
# Import Python libraries.

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score,f1_score

from catboost import CatBoostRegressor , CatBoostClassifier

# Import Datasets

In [10]:
train_data = pd.read_csv("train_encoded_folds.csv").drop("ID",axis=1)
test_data = pd.read_csv("test_encoded.csv").drop("ID",axis=1)


In [11]:
train_data.head(3)

,sex,Aged,Married,TotalDependents,ServiceSpan,MobileService,4GService,CyberProtection,HardwareSupport,TechnicalAssistance,FilmSubscription,QuarterlyPayment,GrandPayment,CustomerAttrition,SettlementProcess_Bank,SettlementProcess_Card,SettlementProcess_Check,SettlementProcess_Electronic,kfold
0,0,0,0,1,48,1,2,0,1,1,0,96.259802,4628.129119,1,0,0,1,0,0
1,1,0,1,0,45,1,2,0,0,0,1,18.876470,922.085998,0,1,0,0,0,6
2,1,1,1,0,59,1,2,1,0,1,0,106.016057,6205.891129,1,0,1,0,0,2


In [12]:
test_data.head(3)

,sex,Aged,Married,TotalDependents,ServiceSpan,MobileService,4GService,CyberProtection,HardwareSupport,TechnicalAssistance,FilmSubscription,QuarterlyPayment,GrandPayment,SettlementProcess_Bank,SettlementProcess_Card,SettlementProcess_Check,SettlementProcess_Electronic
0,1,0,0,0,8,1,2,0,0,0,1,21.299937,151.092519,0,0,1,0
1,0,0,0,0,26,1,1,0,1,0,1,56.609623,1553.276979,0,0,1,0
2,1,0,0,0,66,1,1,0,0,0,1,25.167690,1684.262416,0,1,0,0


# Convert datatype of selected fields.

# Drop the dependent variable from the train/test dataset. 

In [13]:
# ytrain = train_data[['CustomerAttrition','kfold']]
# xtrain = train_data.drop(['kfold','CustomerAttrition'],axis=1)

# concat = pd.concat([xtrain,test_data],axis = 0)
# mini = concat.min(axis=0)
# maxi = concat.max(axis=0)
# concat = (concat - mini)/maxi
# xtrain = concat[:len(xtrain)]
# test_data = concat[len(xtrain):]
# train_data = pd.concat([xtrain,ytrain],axis=1)

# Train your model

In [14]:
def train(model,xtrain,ytrain,xval,yval):
    model.fit(xtrain,ytrain,eval_set=(xval, yval),verbose_eval= True,use_best_model=True)
    y_pred = model.predict(xtrain)
    acc = accuracy_score(ytrain, y_pred)
    f1  = f1_score(ytrain,y_pred)
    print(f'Train acc: {acc} f1: {f1}')
    y_pred = model.predict(xval)
    acc = accuracy_score(yval, y_pred)
    f1  = f1_score(yval,y_pred)
    print(f'Val acc: {acc} f1: {f1}')
    return model,y_pred,acc

def get_pred(model,xtest):
    return model.predict(xtest)

In [15]:
folds = 9

y_test_oof = []
y_pred_oof = []

all_preds = {}

for itr,fold in enumerate(range(folds)):
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True).drop(['kfold'],axis=1)
    X_test  = train_data[train_data.kfold == fold].reset_index(drop=True).drop(['kfold'],axis=1)
    
    y_train = X_train["CustomerAttrition"]
    X_train = X_train.drop("CustomerAttrition",axis=1)
    
    y_test = X_test["CustomerAttrition"]
    X_test = X_test.drop("CustomerAttrition",axis=1)
    
    
    
    print(f"Train size {len(X_train)} | Val size {len(X_test)}")
    print()
    
    model = CatBoostClassifier(iterations=3000, depth=4, learning_rate=0.1 ,l2_leaf_reg=5, loss_function='Logloss',verbose=False,early_stopping_rounds = 200)
    model,y_pred, acc = train(model,X_train,y_train,X_test,y_test)
    
    print(f"Doing Prediction for acc {acc}")
    all_preds[str(acc) + str(itr)] = get_pred(model,test_data)
    
    y_test_oof.extend(y_test)
    y_pred_oof.extend(y_pred)
    
    print()
    


Train size 5634 | Val size 703

0:	learn: 0.6429807	test: 0.6435327	best: 0.6435327 (0)	total: 1.47ms	remaining: 4.42s
1:	learn: 0.6075290	test: 0.6083144	best: 0.6083144 (1)	total: 3.08ms	remaining: 4.62s
2:	learn: 0.5793820	test: 0.5804661	best: 0.5804661 (2)	total: 4.65ms	remaining: 4.65s
3:	learn: 0.5562165	test: 0.5580830	best: 0.5580830 (3)	total: 6.24ms	remaining: 4.67s
4:	learn: 0.5404877	test: 0.5429300	best: 0.5429300 (4)	total: 7.61ms	remaining: 4.56s
5:	learn: 0.5232928	test: 0.5268334	best: 0.5268334 (5)	total: 9.13ms	remaining: 4.55s
6:	learn: 0.5064764	test: 0.5106589	best: 0.5106589 (6)	total: 11.1ms	remaining: 4.75s
7:	learn: 0.4945706	test: 0.5004510	best: 0.5004510 (7)	total: 12.9ms	remaining: 4.83s
8:	learn: 0.4871759	test: 0.4924103	best: 0.4924103 (8)	total: 14.5ms	remaining: 4.82s
9:	learn: 0.4781217	test: 0.4837306	best: 0.4837306 (9)	total: 16.5ms	remaining: 4.94s
10:	learn: 0.4710845	test: 0.4777720	best: 0.4777720 (10)	total: 18.3ms	remaining: 4.97s
11:	learn

126:	learn: 0.3913561	test: 0.4389968	best: 0.4350558 (65)	total: 190ms	remaining: 4.29s
127:	learn: 0.3910469	test: 0.4389893	best: 0.4350558 (65)	total: 191ms	remaining: 4.29s
128:	learn: 0.3907529	test: 0.4389770	best: 0.4350558 (65)	total: 193ms	remaining: 4.29s
129:	learn: 0.3904430	test: 0.4389732	best: 0.4350558 (65)	total: 194ms	remaining: 4.29s
130:	learn: 0.3900946	test: 0.4392589	best: 0.4350558 (65)	total: 196ms	remaining: 4.29s
131:	learn: 0.3898768	test: 0.4396351	best: 0.4350558 (65)	total: 198ms	remaining: 4.29s
132:	learn: 0.3896124	test: 0.4398290	best: 0.4350558 (65)	total: 199ms	remaining: 4.29s
133:	learn: 0.3893408	test: 0.4396355	best: 0.4350558 (65)	total: 201ms	remaining: 4.3s
134:	learn: 0.3889769	test: 0.4393656	best: 0.4350558 (65)	total: 203ms	remaining: 4.3s
135:	learn: 0.3888148	test: 0.4392777	best: 0.4350558 (65)	total: 205ms	remaining: 4.31s
136:	learn: 0.3884419	test: 0.4391530	best: 0.4350558 (65)	total: 207ms	remaining: 4.32s
137:	learn: 0.3881678	t

252:	learn: 0.3631492	test: 0.4418686	best: 0.4350558 (65)	total: 383ms	remaining: 4.16s
253:	learn: 0.3629510	test: 0.4415796	best: 0.4350558 (65)	total: 384ms	remaining: 4.16s
254:	learn: 0.3627333	test: 0.4410623	best: 0.4350558 (65)	total: 386ms	remaining: 4.16s
255:	learn: 0.3625520	test: 0.4411100	best: 0.4350558 (65)	total: 388ms	remaining: 4.16s
256:	learn: 0.3624131	test: 0.4410988	best: 0.4350558 (65)	total: 389ms	remaining: 4.15s
257:	learn: 0.3622064	test: 0.4415751	best: 0.4350558 (65)	total: 391ms	remaining: 4.15s
258:	learn: 0.3619308	test: 0.4421082	best: 0.4350558 (65)	total: 392ms	remaining: 4.15s
259:	learn: 0.3617702	test: 0.4418840	best: 0.4350558 (65)	total: 394ms	remaining: 4.15s
260:	learn: 0.3615223	test: 0.4419956	best: 0.4350558 (65)	total: 396ms	remaining: 4.15s
261:	learn: 0.3613121	test: 0.4420154	best: 0.4350558 (65)	total: 397ms	remaining: 4.15s
262:	learn: 0.3611383	test: 0.4419220	best: 0.4350558 (65)	total: 399ms	remaining: 4.15s
263:	learn: 0.3609353

100:	learn: 0.3973641	test: 0.4472241	best: 0.4465632 (86)	total: 145ms	remaining: 4.15s
101:	learn: 0.3970821	test: 0.4474217	best: 0.4465632 (86)	total: 146ms	remaining: 4.16s
102:	learn: 0.3966785	test: 0.4473354	best: 0.4465632 (86)	total: 148ms	remaining: 4.16s
103:	learn: 0.3962438	test: 0.4475335	best: 0.4465632 (86)	total: 149ms	remaining: 4.16s
104:	learn: 0.3959137	test: 0.4471822	best: 0.4465632 (86)	total: 151ms	remaining: 4.16s
105:	learn: 0.3956691	test: 0.4473392	best: 0.4465632 (86)	total: 153ms	remaining: 4.16s
106:	learn: 0.3953754	test: 0.4470696	best: 0.4465632 (86)	total: 154ms	remaining: 4.17s
107:	learn: 0.3951414	test: 0.4469814	best: 0.4465632 (86)	total: 156ms	remaining: 4.18s
108:	learn: 0.3948200	test: 0.4472041	best: 0.4465632 (86)	total: 158ms	remaining: 4.18s
109:	learn: 0.3945312	test: 0.4470359	best: 0.4465632 (86)	total: 159ms	remaining: 4.18s
110:	learn: 0.3942184	test: 0.4470809	best: 0.4465632 (86)	total: 161ms	remaining: 4.19s
111:	learn: 0.3939101

229:	learn: 0.3653696	test: 0.4531614	best: 0.4465632 (86)	total: 339ms	remaining: 4.09s
230:	learn: 0.3651052	test: 0.4531233	best: 0.4465632 (86)	total: 341ms	remaining: 4.09s
231:	learn: 0.3648095	test: 0.4533041	best: 0.4465632 (86)	total: 342ms	remaining: 4.08s
232:	learn: 0.3645615	test: 0.4535664	best: 0.4465632 (86)	total: 344ms	remaining: 4.08s
233:	learn: 0.3642981	test: 0.4535145	best: 0.4465632 (86)	total: 345ms	remaining: 4.08s
234:	learn: 0.3639855	test: 0.4537356	best: 0.4465632 (86)	total: 347ms	remaining: 4.08s
235:	learn: 0.3637655	test: 0.4537271	best: 0.4465632 (86)	total: 349ms	remaining: 4.08s
236:	learn: 0.3635862	test: 0.4538080	best: 0.4465632 (86)	total: 350ms	remaining: 4.08s
237:	learn: 0.3634569	test: 0.4538896	best: 0.4465632 (86)	total: 352ms	remaining: 4.09s
238:	learn: 0.3632120	test: 0.4542369	best: 0.4465632 (86)	total: 354ms	remaining: 4.09s
239:	learn: 0.3629486	test: 0.4539430	best: 0.4465632 (86)	total: 356ms	remaining: 4.09s
240:	learn: 0.3627463

...

4:	learn: 0.5301072	test: 0.5306756	best: 0.5306756 (4)	total: 7.46ms	remaining: 4.47s
5:	learn: 0.5143127	test: 0.5154296	best: 0.5154296 (5)	total: 9.31ms	remaining: 4.64s
6:	learn: 0.5017078	test: 0.5021914	best: 0.5021914 (6)	total: 11ms	remaining: 4.69s
7:	learn: 0.4906032	test: 0.4914128	best: 0.4914128 (7)	total: 12.9ms	remaining: 4.83s
8:	learn: 0.4821792	test: 0.4827048	best: 0.4827048 (8)	total: 14.9ms	remaining: 4.94s
9:	learn: 0.4742145	test: 0.4752314	best: 0.4752314 (9)	total: 16.6ms	remaining: 4.97s
10:	learn: 0.4681426	test: 0.4686520	best: 0.4686520 (10)	total: 18.4ms	remaining: 5.01s
11:	learn: 0.4640015	test: 0.4646355	best: 0.4646355 (11)	total: 20.2ms	remaining: 5.02s
12:	learn: 0.4592660	test: 0.4594705	best: 0.4594705 (12)	total: 22.2ms	remaining: 5.11s
13:	learn: 0.4548983	test: 0.4548835	best: 0.4548835 (13)	total: 23.8ms	remaining: 5.08s
14:	learn: 0.4520974	test: 0.4522987	best: 0.4522987 (14)	total: 25.3ms	remaining: 5.03s
15:	learn: 0.4493342	test: 0.449101

132:	learn: 0.3930024	test: 0.4207242	best: 0.4194256 (80)	total: 201ms	remaining: 4.33s
133:	learn: 0.3927346	test: 0.4212907	best: 0.4194256 (80)	total: 202ms	remaining: 4.33s
134:	learn: 0.3924675	test: 0.4211773	best: 0.4194256 (80)	total: 204ms	remaining: 4.34s
135:	learn: 0.3920838	test: 0.4212120	best: 0.4194256 (80)	total: 206ms	remaining: 4.34s
136:	learn: 0.3918280	test: 0.4212146	best: 0.4194256 (80)	total: 208ms	remaining: 4.35s
137:	learn: 0.3916059	test: 0.4213246	best: 0.4194256 (80)	total: 210ms	remaining: 4.35s
138:	learn: 0.3914370	test: 0.4212882	best: 0.4194256 (80)	total: 212ms	remaining: 4.36s
139:	learn: 0.3911489	test: 0.4213585	best: 0.4194256 (80)	total: 213ms	remaining: 4.36s
140:	learn: 0.3908341	test: 0.4210566	best: 0.4194256 (80)	total: 215ms	remaining: 4.37s
141:	learn: 0.3906577	test: 0.4209419	best: 0.4194256 (80)	total: 217ms	remaining: 4.37s
142:	learn: 0.3903156	test: 0.4208681	best: 0.4194256 (80)	total: 218ms	remaining: 4.36s
143:	learn: 0.3899309

258:	learn: 0.3655444	test: 0.4219258	best: 0.4194256 (80)	total: 394ms	remaining: 4.17s
259:	learn: 0.3654148	test: 0.4220081	best: 0.4194256 (80)	total: 396ms	remaining: 4.17s
260:	learn: 0.3651902	test: 0.4220836	best: 0.4194256 (80)	total: 397ms	remaining: 4.17s
261:	learn: 0.3650033	test: 0.4219407	best: 0.4194256 (80)	total: 399ms	remaining: 4.17s
262:	learn: 0.3648735	test: 0.4219881	best: 0.4194256 (80)	total: 400ms	remaining: 4.17s
263:	learn: 0.3647128	test: 0.4218754	best: 0.4194256 (80)	total: 402ms	remaining: 4.17s
264:	learn: 0.3645040	test: 0.4217756	best: 0.4194256 (80)	total: 403ms	remaining: 4.16s
265:	learn: 0.3642267	test: 0.4216692	best: 0.4194256 (80)	total: 405ms	remaining: 4.16s
266:	learn: 0.3641344	test: 0.4217086	best: 0.4194256 (80)	total: 407ms	remaining: 4.16s
267:	learn: 0.3637220	test: 0.4220678	best: 0.4194256 (80)	total: 408ms	remaining: 4.16s
268:	learn: 0.3635302	test: 0.4222402	best: 0.4194256 (80)	total: 410ms	remaining: 4.16s
269:	learn: 0.3633035

88:	learn: 0.4057433	test: 0.4210087	best: 0.4207205 (77)	total: 130ms	remaining: 4.25s
89:	learn: 0.4053178	test: 0.4210981	best: 0.4207205 (77)	total: 132ms	remaining: 4.25s
90:	learn: 0.4051340	test: 0.4210870	best: 0.4207205 (77)	total: 133ms	remaining: 4.26s
91:	learn: 0.4047578	test: 0.4216407	best: 0.4207205 (77)	total: 135ms	remaining: 4.26s
92:	learn: 0.4046298	test: 0.4217562	best: 0.4207205 (77)	total: 136ms	remaining: 4.26s
93:	learn: 0.4043594	test: 0.4219463	best: 0.4207205 (77)	total: 138ms	remaining: 4.26s
94:	learn: 0.4039745	test: 0.4216188	best: 0.4207205 (77)	total: 139ms	remaining: 4.26s
95:	learn: 0.4035439	test: 0.4214292	best: 0.4207205 (77)	total: 141ms	remaining: 4.26s
96:	learn: 0.4030926	test: 0.4214939	best: 0.4207205 (77)	total: 142ms	remaining: 4.26s
97:	learn: 0.4026990	test: 0.4219381	best: 0.4207205 (77)	total: 144ms	remaining: 4.25s
98:	learn: 0.4024048	test: 0.4221139	best: 0.4207205 (77)	total: 145ms	remaining: 4.26s
99:	learn: 0.4022092	test: 0.422

211:	learn: 0.3735886	test: 0.4254480	best: 0.4207205 (77)	total: 324ms	remaining: 4.26s
212:	learn: 0.3733507	test: 0.4257433	best: 0.4207205 (77)	total: 326ms	remaining: 4.26s
213:	learn: 0.3731177	test: 0.4258249	best: 0.4207205 (77)	total: 327ms	remaining: 4.26s
214:	learn: 0.3728630	test: 0.4257807	best: 0.4207205 (77)	total: 329ms	remaining: 4.26s
215:	learn: 0.3726654	test: 0.4259257	best: 0.4207205 (77)	total: 331ms	remaining: 4.26s
216:	learn: 0.3725403	test: 0.4262066	best: 0.4207205 (77)	total: 332ms	remaining: 4.26s
217:	learn: 0.3723012	test: 0.4266815	best: 0.4207205 (77)	total: 334ms	remaining: 4.26s
218:	learn: 0.3720615	test: 0.4268729	best: 0.4207205 (77)	total: 335ms	remaining: 4.26s
219:	learn: 0.3719758	test: 0.4268555	best: 0.4207205 (77)	total: 337ms	remaining: 4.25s
220:	learn: 0.3717436	test: 0.4269544	best: 0.4207205 (77)	total: 338ms	remaining: 4.25s
221:	learn: 0.3714503	test: 0.4267665	best: 0.4207205 (77)	total: 340ms	remaining: 4.25s
222:	learn: 0.3713426

In [16]:
oof = accuracy_score(y_test_oof, y_pred_oof)
print('OOF acc: %f' % oof)

sample_sub = pd.read_csv("Sample Submission.csv")
base = np.zeros(len(test_data))
for val in all_preds.values():
    base += np.array(val)


sample_sub['CustomerAttrition'] = 1*(base > (folds//2))

sample_sub.replace({1:'Yes', 0:'No'}, inplace=True)
sample_sub.to_csv("submission_scaled.csv",index=False)

OOF acc: 0.799116


In [ ]:
sample_sub

In [ ]:
"""


For CrossEntropy : depth = 4 , lr = 0.1, l2_leaf_reg=5 loss : 0.4224531
For LogLoss      : depth = 4 , lr = 0.1, l2_leaf_reg=5 loss : 0.4224531


unscaled gave 0.799116 oof

"""

In [ ]:
# RMSE, MultiRMSE, MAE, Quantile, LogLinQuantile, Poisson, MAPE, Lq or custom objective object

In [ ]:
# get_pred(DTClassifier,test_data)

In [ ]:
# cols = np.array(train_data.columns)[1:-1]
# impt = DTClassifier.feature_importances_

# df = pd.DataFrame([cols,impt]).T.rename({0:"Feature",1:"Importance"},axis=1)

In [ ]:
# def rank_data(sub):
#     sub = pd.DataFrame(base)[0]
#     sub = sub.rank() / sub.rank().max()
#     return sub

In [ ]:
grid = {'learning_rate': [0.01,0.03,0.05,0.1,1.0],
    'depth': [4, 6,8 ,10],
    'l2_leaf_reg': [1, 3, 5, 7, 9]}

ytrain = train_data['CustomerAttrition']
xtrain = train_data.drop(['kfold','CustomerAttrition'],axis=1)

model = CatBoostClassifier(iterations=1500,loss_function='Logloss',verbose=False)
grid_search_result = model.grid_search(grid, 
                                   X=xtrain, 
                                   y=ytrain, 
                                   stratified = True,
                                   cv = 9,
                                   plot=True)